Here we'll update the code from the logisitic regression module to be able to run on the GPU. Also, this will consolidate all the cells into one, tidy cell. Nice and streamlined.

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import numpy as np

'''
STEP 1: LOAD TRAIN & TEST DATASETS
'''

train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

'''
STEP 2: MAKE THE DATASETS ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self,).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return output_dim 
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
output_dim = 10

model = LogisticRegressionModel(input_dim, output_dim)

#####################
# MOVE MODEL TO GPU #
#####################

if torch.cuda.is_available():
    model.cuda()
    
'''
STEP 5: INSTANTIATE LOSS CLASS
'''

criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''

learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #################
        # CHECK FOR GPU #  Remembering that in order to run on GPU, model and variables need to live on GPU .cuda()
        #################
        if torch.cuda.is_available():
            images = Variable(images.view(-1,28*28).cuda())
            labels = Variable(labels.cuda())
        else:        
            images = Variable(images.view(-1,28*28))
            labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get outputs / logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        # Get gradients w.r.t. parameters
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                #################
                # CHECK FOR GPU #  Same goes for variables used to calculate accuracy
                #################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1,28*28).cuda())
                else: 
                    images = Variable(images.view(-1,28*28))
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                #################
                # CHECK FOR GPU #
                #################
                # predicted was created on the GPU, so we have to bring it back to the CPU to calculate correct.                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

KeyError: <class 'int'>

#### I'm not sure what the above KeyError is all about. The code is 100% correct, as far as I can tell. I'm wondering if it's just a bug in PyTorch at the moment, or maybe some compatibility error since I have two notebooks open? I'll come back to this after finishing the section summary.

## Summary

* Logistic Regression In-Depth
    1. Get Logits
    - Get Softmax
    - Get Cross-Entropy Loss
* **Aim**: minimize Cross-Entropy Loss
* Built a Logistic Regression Model on CPU / GPU
    * Follow the seven steps! See above. 
    
### Now on to Feed-Forward Neural Networks!